In [1]:
# 本代码是探究指定时间进入指定地点的人群

In [2]:
# 思路
# 打算使用字典的方式存储，按每天的每个时间段进行分类

# 导入模块

In [3]:
import pandas as pd
import os
from os import path

In [4]:
import warnings

warnings.filterwarnings(action='ignore')

In [5]:
from datetime import time

# 定义函数

In [6]:
def get_data(file_name):
    '''
    传入的是文件路径
    读取并对数据进行处理
    '''
    file_extension = path.splitext(file_name)[1]
    if file_extension == '.csv':
        data = pd.read_csv(file_name, encoding='gbk')
    elif file_extension == '.xlsx':
        data = pd.read_excel(file_name, encoding='gbk')
    drop_col = ['imei_', 'area_', 'msisdn_']
    data.drop(drop_col, axis=1, inplace=True)
    col = data.columns
    for each in col:
        if data[each].dtype == object:
            data[each] = data[each].str.replace('\t', '')
    print('data load successfully')
    return data

In [31]:
def find_cycle(data1):
    data1.index = pd.to_datetime(data.update_time_)
    # 数据存储
    # 刚才似乎每考虑小时

    d = {}
    for i, j in data1.groupby([
        data1.index.day, data1.index.hour, 'device_id_'
    ]).imsi_: # 可以发现，这样编写可以找出每天某时间，出现的用户
        day = i[0]
        hour = i[1]
        device = i[2]
        if d.get((day, hour)) == None:
            d[(day, hour)] = {}
        d[(day, hour)].update({device:set(j.to_list())})
        
    # 开始集合运算
    res = {}
    flag = 1
    device = ['DX-SZSC001', 'LT-SZSC001', 'YD-SZSC001']
    for i in device:
        for day in d.keys():
            if flag:
                base = d[day][i]
                flag = 0
            base &= d[day][i]
            if res.get(day) == None:
                res[day] = {}
            res[day].update({i:base})
        flag = 1
    return res

In [34]:
def main():
    os.listdir('../Project Data/')
    os.chdir('../Project Data/') # 切换到数据所在目录
    data = get_data('20190524-001.csv')
    return find_cycle(data)

# 运行代码

In [42]:
res = main()

# 代码调试

In [7]:
os.listdir('../Project Data/')

os.chdir('../Project Data/') # 切换到数据所在目录

In [8]:
data = get_data('20190524-001.csv')

data load successfully


In [9]:
data.head()

,imsi_,update_time_,lac_,cellid_,status_,ap_type_,device_id_,eventid_
0,460110413044304,2019-05-08 20:09:29,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
1,460110413042577,2019-05-08 20:09:41,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
2,460110425023300,2019-05-08 20:10:14,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
3,460110714141865,2019-05-08 20:10:40,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
4,460016692088409,2019-05-08 20:08:55,0,0,拒绝,FDD-LTE,LT-SZSC001,Normal Lau


In [10]:
data1 = data.copy()

In [11]:
data1.index = pd.to_datetime(data.update_time_)

In [12]:
data1.head()

,imsi_,update_time_,lac_,cellid_,status_,ap_type_,device_id_,eventid_
update_time_,,,,,,,,
2019-05-08 20:09:29,460110413044304,2019-05-08 20:09:29,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
2019-05-08 20:09:41,460110413042577,2019-05-08 20:09:41,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
2019-05-08 20:10:14,460110425023300,2019-05-08 20:10:14,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
2019-05-08 20:10:40,460110714141865,2019-05-08 20:10:40,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
2019-05-08 20:08:55,460016692088409,2019-05-08 20:08:55,0,0,拒绝,FDD-LTE,LT-SZSC001,Normal Lau


In [13]:
# 选取某个时间段的数据
data1.loc[time(8,0):time(10, 0)].device_id_.value_counts()

LT-SZSC001    19548
YD-SZSC001    17894
DX-SZSC001     5975
Name: device_id_, dtype: int64

In [25]:
# 数据存储
d = {}
for i, j in data1.groupby([
    data1.index.day, 'device_id_'
]).imsi_: # 可以发现，这样编写可以找出每天某时间，出现的用户
    day = i[0]
    device = i[1]
    if d.get(day) == None: # 使用字典的get方法，如果元素不存在，则返回None
        d[day] = {}
    d[day].update({device:set(j)}) # 为什么叠加不了

In [22]:
d.get(2) == None #

True

In [27]:
for i in range(8, 25):
    print(d[i].keys())

dict_keys(['DX-SZSC001', 'LT-SZSC001', 'YD-SZSC001'])
dict_keys(['DX-SZSC001', 'LT-SZSC001', 'YD-SZSC001'])
dict_keys(['DX-SZSC001', 'LT-SZSC001', 'YD-SZSC001'])
dict_keys(['DX-SZSC001', 'LT-SZSC001', 'YD-SZSC001'])
dict_keys(['DX-SZSC001', 'LT-SZSC001', 'YD-SZSC001'])
dict_keys(['DX-SZSC001', 'LT-SZSC001', 'YD-SZSC001'])
dict_keys(['DX-SZSC001', 'LT-SZSC001', 'YD-SZSC001'])
dict_keys(['DX-SZSC001', 'LT-SZSC001', 'YD-SZSC001'])
dict_keys(['DX-SZSC001', 'LT-SZSC001', 'YD-SZSC001'])
dict_keys(['DX-SZSC001', 'LT-SZSC001', 'YD-SZSC001'])
dict_keys(['DX-SZSC001', 'LT-SZSC001', 'YD-SZSC001'])
dict_keys(['DX-SZSC001', 'LT-SZSC001', 'YD-SZSC001'])
dict_keys(['DX-SZSC001', 'LT-SZSC001', 'YD-SZSC001'])
dict_keys(['DX-SZSC001', 'LT-SZSC001', 'YD-SZSC001'])
dict_keys(['DX-SZSC001', 'LT-SZSC001', 'YD-SZSC001'])
dict_keys(['DX-SZSC001', 'LT-SZSC001', 'YD-SZSC001'])
dict_keys(['DX-SZSC001', 'LT-SZSC001', 'YD-SZSC001'])


In [29]:
# 开始集合运算

res = {}
flag = 1
device = ['DX-SZSC001', 'LT-SZSC001', 'YD-SZSC001']
for i in device:
    for day in d.keys():
        if flag:
            base = d[day][i]
            flag = 0
        base &= d[day][i]
        if res.get(day) == None:
            res[day] = {}
        res[day].update({i:base})
    flag = 1

In [38]:
# 刚才似乎每考虑小时
# 数据存储
d = {}
for i, j in data1.groupby([
    data1.index.day, data1.index.hour, 'device_id_'
]).imsi_: # 可以发现，这样编写可以找出每天某时间，出现的用户
    day = i[0]
    hour = i[1]
    device = i[2]
    if d.get((day, hour)) == None:
        d[(day, hour)] = {}
    d[(day, hour)].update({device:set(j.to_list())})

In [41]:
# 开始定位，某个时间段出现什么人
d[(8, 20)].keys()# 时间区间，表示9号，10号，11号

dict_keys(['DX-SZSC001', 'LT-SZSC001', 'YD-SZSC001'])

In [32]:
res = None
day = ''
for i, j in d.items():
    if not res:
        day = ','.join([day, str(i)])
        res = j
    else:
        day = ','.join([day, str(i)])
        res &= j
print(f'用户：{res} 在这个月的 {day[1:]} 天的 8-10 点出现了')

用户：{460110267853698, 460000211720632, 460002406691111, 460029571752973, 460110412995758, 460014926623352, 460004202121725} 在这个月的 9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24 天的 8-10 点出现了


In [23]:
s = ''
','.join(['1', s])

'1,'